In [1]:
import numpy as np
import os
import glob
import pandas as pd

In [4]:
dir_name = glob.glob(r'../../00_Kaneki/results/*.csv')
all_data=np.zeros((len(dir_name),491))

for index, name in enumerate(dir_name):
    path = '../../00_Kaneki/results/' + str(index) + '.csv'
    data = pd.read_csv(path,header = None)
    data1=np.array(data)
    all_data[index,:]=data1
print all_data

[[4.4463e+02 3.5672e-02 5.8109e+00 ... 9.0493e-01 7.8880e-01 3.8257e-01]
 [3.2388e+02 5.5815e-03 5.5475e+00 ... 6.5424e-01 1.2726e+00 1.1577e+00]
 [3.5204e+02 1.5560e-02 5.5999e+00 ... 8.7005e-01 7.9320e-01 8.2680e-01]
 ...
 [2.7530e+02 8.9307e-02 5.4169e+00 ... 2.5477e+00 2.0142e+00 1.6806e+00]
 [5.0100e+02 2.8378e-01 6.0655e+00 ... 3.2340e+00 4.7615e+00 3.7687e+00]
 [4.0921e+02 9.0453e-03 5.8048e+00 ... 1.1404e+00 2.9595e-01 1.2543e+00]]


In [5]:
all_data.shape

(19658, 491)

In [6]:
np.save('features.npy',all_data)

In [7]:
#-*- coding:utf-8
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Convolution1D, Conv2D, MaxPooling2D, MaxPooling1D, LSTM, Embedding
from keras.optimizers import SGD
import keras.backend as K
import random
import re
import tensorflow as tf
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

config_proto = tf.ConfigProto(log_device_placement=0,allow_soft_placement=0)
config_proto.gpu_options.allow_growth = True
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

Using TensorFlow backend.


In [8]:
X_train = np.load('../../20_Galaxy/ANGRY/2HXT-train.npy')
Y_train = np.load('../../20_Galaxy/ANGRY/2HXT-train_label.npy')

X_valid = np.load('../../20_Galaxy/ANGRY/2HXT-val.npy')
Y_valid = np.load('../../20_Galaxy/ANGRY/2HXT-val_label.npy')

In [9]:
one_hot = preprocessing.OneHotEncoder(sparse = False)
y_train = one_hot.fit_transform(Y_train)
y_valid = one_hot.fit_transform(Y_valid)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [12]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_3/concat:0", shape=(?, 169, 60), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 290s 19ms/step - loss: 0.6121 - acc: 0.6450 - val_loss: 0.4631 - val_acc: 0.8322
Epoch 2/50
15658/15658 [==============================] - 290s 19ms/step - loss: 0.4749 - acc: 0.7724 - val_loss: 0.4843 - val_acc: 0.8698
Epoch 3/50
15658/15658 [==============================] - 287s 18ms/step - loss: 0.4307 - acc: 0.8014 - val_loss: 0.5738 - val_acc: 0.8002
Epoch 4/50
15658/15658 [==============================] - 288s 18ms/step - loss: 0.3994 - acc: 0.8211 - val_loss: 0.4110 - val_acc: 0.8950
Epoch 5/50
15658/15658 [==============================] - 286s 18ms/step - loss: 0.3737 - acc: 0.8385 - val_loss: 0.4265 - val_acc: 0.9002
Epoch 6/50
15658/15658 [==============================] - 288s 18ms/step - loss: 0.3598 - acc: 0.8455 - val_loss: 0.4294 - val_acc: 0.8990
Epoch 7/50
15658/15658 [=========================

In [39]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut1(x):
    x = x[:,:12*3640]
    return x

def cut2(x):
    x = x[:,12*3640:]
    x = x.reshape(491,1)
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
        inputs = Lambda(cut1)(model_input)
        print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    
#     feature_list2 = []
#     feature_list2.append(flatten1)
#     inputs2 = Lambda(cut1)(model_input)
#     inputs3 = Flatten()(inputs2)
    
#     print inputs3
#     print flatten1
#     feature_list2.append(inputs3)
    
#     flatten2 = Concatenate(axis=1)(feature_list2)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS+491,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
print X_train.shape
xtrain = X_train.reshape(-1,12*3640)
xvalid = X_valid.reshape(-1,12*3640)
print xtrain.shape
xtrain = pd.concat([pd.DataFrame(xtrain),pd.DataFrame(all_data[:15658])], axis=1)
xvalid = pd.concat([pd.DataFrame(xvalid),pd.DataFrame(all_data[15658:])], axis=1)
xtrain = np.array(xtrain)
xvalid = np.array(xvalid)
print xtrain.shape
xtrain = xtrain.reshape(-1,12*3640+491,1)
xvalid = xvalid.reshape(-1,12*3640+491,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("lambda_73/strided_slice:0", shape=(?, 43680, 1), dtype=float32)
Tensor("lambda_74/strided_slice:0", shape=(?, 43680, 1), dtype=float32)
Tensor("concatenate_51/concat:0", shape=(?, 169, 60), dtype=float32)
Tensor("flatten_30/Reshape:0", shape=(?, ?), dtype=float32)
Tensor("flatten_29/Reshape:0", shape=(?, ?), dtype=float32)
(15658, 12, 3640)
(15658, 43680)
(15658, 44171)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 298s 19ms/step - loss: 0.6233 - acc: 0.6280 - val_loss: 0.4762 - val_acc: 0.8135
Epoch 2/50
15658/15658 [==============================] - 294s 19ms/step - loss: 0.4768 - acc: 0.7706 - val_loss: 0.4614 - val_acc: 0.8450
Epoch 3/50
15658/15658 [==============================] - 294s 19ms/step - loss: 0.4279 - acc: 0.8046 - val_loss: 0.4517 - val_acc: 0.8530
Epoch 4/50
15658/15658 [==============================] - 295s 19ms/step - loss: 0.4009 - acc: 0.8208 - val_loss: 0.4494 - val_acc: 0.8900
Epoch 5/50
156

In [48]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut1(x):
    x = x[:,:12*3640]
    return x

def cut2(x):
    x = x[:,12*3640:]
    x = x.reshape(491,1)
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
        inputs = Lambda(cut1)(model_input)
        print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    
    feature_list2 = []
    feature_list2.append(flatten1)
    inputs2 = Lambda(cut1)(model_input)
    inputs3 = Flatten()(inputs2)
    
#     print inputs3
#     print flatten1
    feature_list2.append(inputs3)
    
    flatten2 = Concatenate(axis=1)(feature_list2)
    
    print flatten2
    flatten3 = Dense(8, kernel_initializer='glorot_uniform', activation='softmax')(flatten2)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten3)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS+491,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
print X_train.shape
xtrain = X_train.reshape(-1,12*3640)
xvalid = X_valid.reshape(-1,12*3640)
print xtrain.shape
xtrain = pd.concat([pd.DataFrame(xtrain),pd.DataFrame(all_data[:15658])], axis=1)
xvalid = pd.concat([pd.DataFrame(xvalid),pd.DataFrame(all_data[15658:])], axis=1)
xtrain = np.array(xtrain)
xvalid = np.array(xvalid)
print xtrain.shape
xtrain = xtrain.reshape(-1,12*3640+491,1)
xvalid = xvalid.reshape(-1,12*3640+491,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("lambda_100/strided_slice:0", shape=(?, 43680, 1), dtype=float32)
Tensor("lambda_101/strided_slice:0", shape=(?, 43680, 1), dtype=float32)
Tensor("concatenate_69/concat:0", shape=(?, 169, 60), dtype=float32)
Tensor("concatenate_70/concat:0", shape=(?, ?), dtype=float32)
(15658, 12, 3640)
(15658, 43680)
(15658, 44171)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 302s 19ms/step - loss: 0.6915 - acc: 0.5197 - val_loss: 0.6964 - val_acc: 0.5040
Epoch 2/50
15658/15658 [==============================] - 289s 18ms/step - loss: 0.6717 - acc: 0.5855 - val_loss: 0.6489 - val_acc: 0.6285
Epoch 3/50
15658/15658 [==============================] - 291s 19ms/step - loss: 0.6041 - acc: 0.6736 - val_loss: 0.5435 - val_acc: 0.7688
Epoch 4/50
15658/15658 [==============================] - 294s 19ms/step - loss: 0.5060 - acc: 0.7581 - val_loss: 0.4470 - val_acc: 0.8322
Epoch 5/50
15658/15658 [==============================] - 290s 19ms/s

In [46]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut1(x):
    x = x[:,:12*3640]
    return x

def cut2(x):
    x = x[:,12*3640:]
    x = x.reshape(491,1)
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
        inputs = Lambda(cut1)(model_input)
        print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    
    feature_list2 = []
    feature_list2.append(flatten1)
    inputs2 = Lambda(cut1)(model_input)
    inputs3 = Flatten()(inputs2)
    
#     print inputs3
#     print flatten1
    feature_list2.append(inputs3)
    
    flatten2 = Concatenate(axis=1)(feature_list2)
    
    print flatten2
    flatten3 = Dense(32, kernel_initializer='glorot_uniform', activation='softmax')(flatten2)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten3)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS+491,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
print X_train.shape
xtrain = X_train.reshape(-1,12*3640)
xvalid = X_valid.reshape(-1,12*3640)
print xtrain.shape
xtrain = pd.concat([pd.DataFrame(xtrain),pd.DataFrame(all_data[:15658])], axis=1)
xvalid = pd.concat([pd.DataFrame(xvalid),pd.DataFrame(all_data[15658:])], axis=1)
xtrain = np.array(xtrain)
xvalid = np.array(xvalid)
print xtrain.shape
xtrain = xtrain.reshape(-1,12*3640+491,1)
xvalid = xvalid.reshape(-1,12*3640+491,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("lambda_94/strided_slice:0", shape=(?, 43680, 1), dtype=float32)
Tensor("lambda_95/strided_slice:0", shape=(?, 43680, 1), dtype=float32)
Tensor("concatenate_65/concat:0", shape=(?, 169, 60), dtype=float32)
Tensor("concatenate_66/concat:0", shape=(?, ?), dtype=float32)
(15658, 12, 3640)
(15658, 43680)
(15658, 44171)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 301s 19ms/step - loss: 0.6917 - acc: 0.5276 - val_loss: 0.6959 - val_acc: 0.5000
Epoch 2/50
15658/15658 [==============================] - 296s 19ms/step - loss: 0.6900 - acc: 0.5293 - val_loss: 0.6950 - val_acc: 0.4998
Epoch 3/50
15658/15658 [==============================] - 295s 19ms/step - loss: 0.6880 - acc: 0.5317 - val_loss: 0.6943 - val_acc: 0.4990
Epoch 4/50
15658/15658 [==============================] - 293s 19ms/step - loss: 0.6842 - acc: 0.5423 - val_loss: 0.6887 - val_acc: 0.5105
Epoch 5/50
15658/15658 [==============================] - 291s 19ms/ste

In [49]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut1(x):
    x = x[:,:12*3640]
    return x

def cut2(x):
    x = x[:,12*3640:]
    x = x.reshape(491,1)
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
        inputs = Lambda(cut1)(model_input)
        print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    
    feature_list2 = []
    feature_list2.append(flatten1)
    inputs2 = Lambda(cut1)(model_input)
    inputs3 = Flatten()(inputs2)
    
#     print inputs3
#     print flatten1
    feature_list2.append(inputs3)
    
    flatten2 = Concatenate(axis=1)(feature_list2)
    
    print flatten2
    flatten3 = Dense(4, kernel_initializer='glorot_uniform', activation='softmax')(flatten2)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten3)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS+491,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
print X_train.shape
xtrain = X_train.reshape(-1,12*3640)
xvalid = X_valid.reshape(-1,12*3640)
print xtrain.shape
xtrain = pd.concat([pd.DataFrame(xtrain),pd.DataFrame(all_data[:15658])], axis=1)
xvalid = pd.concat([pd.DataFrame(xvalid),pd.DataFrame(all_data[15658:])], axis=1)
xtrain = np.array(xtrain)
xvalid = np.array(xvalid)
print xtrain.shape
xtrain = xtrain.reshape(-1,12*3640+491,1)
xvalid = xvalid.reshape(-1,12*3640+491,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("lambda_103/strided_slice:0", shape=(?, 43680, 1), dtype=float32)
Tensor("lambda_104/strided_slice:0", shape=(?, 43680, 1), dtype=float32)
Tensor("concatenate_71/concat:0", shape=(?, 169, 60), dtype=float32)
Tensor("concatenate_72/concat:0", shape=(?, ?), dtype=float32)
(15658, 12, 3640)
(15658, 43680)
(15658, 44171)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 302s 19ms/step - loss: 0.6959 - acc: 0.5206 - val_loss: 0.6891 - val_acc: 0.5160
Epoch 2/50
15658/15658 [==============================] - 293s 19ms/step - loss: 0.6616 - acc: 0.5823 - val_loss: 0.6770 - val_acc: 0.5595
Epoch 3/50
15658/15658 [==============================] - 294s 19ms/step - loss: 0.5841 - acc: 0.6966 - val_loss: 0.5201 - val_acc: 0.7863
Epoch 4/50
15658/15658 [==============================] - 294s 19ms/step - loss: 0.4919 - acc: 0.7696 - val_loss: 0.4896 - val_acc: 0.8008
Epoch 5/50
15658/15658 [==============================] - 293s 19ms/s

In [57]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut1(x):
    x = x[:,:12*3640]
    return x

def cut2(x):
    x = x[:,12*3640:]
    x = x.reshape(491,1)
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
        inputs = Lambda(cut1)(model_input)
        print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    
    feature_list2 = []
    feature_list2.append(flatten1)
    inputs2 = Lambda(cut1)(model_input)
    inputs3 = Flatten()(inputs2)
    
#     print inputs3
#     print flatten1
    feature_list2.append(inputs3)
    
    flatten2 = Concatenate(axis=1)(feature_list2)
    
    print flatten2
    flatten3 = Dense(12, kernel_initializer='glorot_uniform', activation='softmax')(flatten2)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten3)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS+491,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
print X_train.shape
xtrain = X_train.reshape(-1,12*3640)
xvalid = X_valid.reshape(-1,12*3640)
print xtrain.shape
xtrain = pd.concat([pd.DataFrame(xtrain),pd.DataFrame(all_data[:15658])], axis=1)
xvalid = pd.concat([pd.DataFrame(xvalid),pd.DataFrame(all_data[15658:])], axis=1)
xtrain = np.array(xtrain)
xvalid = np.array(xvalid)
print xtrain.shape
xtrain = xtrain.reshape(-1,12*3640+491,1)
xvalid = xvalid.reshape(-1,12*3640+491,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("lambda_127/strided_slice:0", shape=(?, 43680, 1), dtype=float32)
Tensor("lambda_128/strided_slice:0", shape=(?, 43680, 1), dtype=float32)
Tensor("concatenate_87/concat:0", shape=(?, 169, 60), dtype=float32)
Tensor("concatenate_88/concat:0", shape=(?, ?), dtype=float32)
(15658, 12, 3640)
(15658, 43680)
(15658, 44171)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 300s 19ms/step - loss: 0.6917 - acc: 0.5273 - val_loss: 0.6936 - val_acc: 0.4947
Epoch 2/50
15658/15658 [==============================] - 291s 19ms/step - loss: 0.6789 - acc: 0.5605 - val_loss: 0.6866 - val_acc: 0.5312
Epoch 3/50
15658/15658 [==============================] - 289s 18ms/step - loss: 0.6405 - acc: 0.6282 - val_loss: 0.6037 - val_acc: 0.6777
Epoch 4/50
15658/15658 [==============================] - 289s 18ms/step - loss: 0.5614 - acc: 0.7112 - val_loss: 0.4961 - val_acc: 0.8037
Epoch 5/50
15658/15658 [==============================] - 289s 18ms/s

In [58]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut1(x):
    x = x[:,:12*3640]
    return x

def cut2(x):
    x = x[:,12*3640:]
    x = x.reshape(491,1)
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
        inputs = Lambda(cut1)(model_input)
        print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    
    feature_list2 = []
    feature_list2.append(flatten1)
    inputs2 = Lambda(cut1)(model_input)
    inputs3 = Flatten()(inputs2)
    
#     print inputs3
#     print flatten1
    feature_list2.append(inputs3)
    
    flatten2 = Concatenate(axis=1)(feature_list2)
    
    print flatten2
    flatten3 = Dense(12, kernel_initializer='glorot_uniform', activation='softmax')(flatten2)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten3)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS+491,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.002, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
print X_train.shape
xtrain = X_train.reshape(-1,12*3640)
xvalid = X_valid.reshape(-1,12*3640)
print xtrain.shape
xtrain = pd.concat([pd.DataFrame(xtrain),pd.DataFrame(all_data[:15658])], axis=1)
xvalid = pd.concat([pd.DataFrame(xvalid),pd.DataFrame(all_data[15658:])], axis=1)
xtrain = np.array(xtrain)
xvalid = np.array(xvalid)
print xtrain.shape
xtrain = xtrain.reshape(-1,12*3640+491,1)
xvalid = xvalid.reshape(-1,12*3640+491,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("lambda_130/strided_slice:0", shape=(?, 43680, 1), dtype=float32)
Tensor("lambda_131/strided_slice:0", shape=(?, 43680, 1), dtype=float32)
Tensor("concatenate_89/concat:0", shape=(?, 169, 60), dtype=float32)
Tensor("concatenate_90/concat:0", shape=(?, ?), dtype=float32)
(15658, 12, 3640)
(15658, 43680)
(15658, 44171)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 296s 19ms/step - loss: 0.6917 - acc: 0.5213 - val_loss: 0.6950 - val_acc: 0.4938
Epoch 2/50
15658/15658 [==============================] - 288s 18ms/step - loss: 0.6867 - acc: 0.5334 - val_loss: 0.6945 - val_acc: 0.4925
Epoch 3/50
15658/15658 [==============================] - 289s 18ms/step - loss: 0.6819 - acc: 0.5526 - val_loss: 0.6939 - val_acc: 0.5035
Epoch 4/50
15658/15658 [==============================] - 289s 18ms/step - loss: 0.6755 - acc: 0.5730 - val_loss: 0.6893 - val_acc: 0.5108
Epoch 5/50
15658/15658 [==============================] - 290s 19ms/s